In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import pprint
import os
import csv


In [2]:
# Read from the csv file that has the trends of pollutants over the years
# Skip the first row as it's empty get the data from column 2 through 32  
ghg_emissions_and_sinks = pd.read_csv('Table 2-2.csv', skiprows = 1, usecols = [col for col in range(1,32)])

# Transpose the data frame 
ghg_emissions_and_sinks = ghg_emissions_and_sinks.T

# Make the first row as the column header
ghg_emissions_and_sinks.columns = ghg_emissions_and_sinks.iloc[0]

# Set the years as index
ghg_emissions_and_sinks =  ghg_emissions_and_sinks.set_index('Gas/Source')
#ghg_emmisions_and_sinks

# Drop column 1 as it's a blank column
ghg_emissions_and_sinks = ghg_emissions_and_sinks.drop('Gas/Source', axis = 0)
ghg_emissions_and_sinks

Unnamed: 1,CO2,Fossil Fuel Combustion,Transportation,Electric Power,Industrial,Residential,Commercial,U.S. Territories,Non-Energy Use of Fuels,Petroleum Systems,...,"Unspecified Mix of HFCs, PFCs, SF6, and NF3",Electronics Industry,NaN,Notes: Totals may not sum due to independent rounding. Parentheses indicate negative values or sequestration.,+ Does not exceed 0.5 kt.,M - Mixture of multiple gases,"a Emissions from Wood Biomass, Ethanol, and Biodiesel Consumption are not included specifically in summing Energy sector totals. Net carbon fluxes from changes in biogenic carbon reservoirs are accounted for in the estimates for LULUCF.",b Emissions from International Bunker Fuels are not included in totals.,c LULUCF emissions of LULUCF CH4 and N2O are reported separately from gross emissions totals. Refer to Table 2-8 for a breakout of emissions and removals for LULUCF by gas and source category.,d Small amounts of PFC emissions also result from this source.
Gas/Source,,,,,,,,,,,,,,,,,,,,,
1990,"5,113,455","4,731,466","1,469,116","1,819,951","853,808","338,578","228,298","21,715","112,766","9,709",...,M,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1991,"5,057,931","4,680,858","1,422,498","1,817,359","839,111","347,719","232,665","21,505","118,155","10,000",...,M,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1992,"5,167,481","4,779,991","1,477,783","1,830,688","866,425","353,989","228,049","23,059","124,568","9,962",...,M,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993,"5,267,343","4,893,308","1,508,743","1,906,172","865,292","366,091","222,991","24,020","113,501","10,021",...,M,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1994,"5,358,824","4,964,870","1,554,070","1,930,468","872,851","357,050","224,700","25,732","124,026","10,274",...,M,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1995,"5,421,502","5,022,522","1,580,796","1,947,173","889,939","353,134","227,789","23,691","121,447","10,739",...,M,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996,"5,610,582","5,214,331","1,627,475","2,020,243","922,940","383,474","237,558","22,641","118,574","11,153",...,M,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997,"5,686,465","5,274,912","1,643,618","2,087,520","917,785","365,033","237,819","23,137","129,352","11,559",...,M,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998,"5,731,050","5,320,581","1,679,024","2,176,555","890,758","331,610","220,118","22,516","134,503","11,476",...,M,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
ghg_emissions_and_sinks.index.name = 'year'
ghg_emissions_and_sinks

Unnamed: 1,CO2,Fossil Fuel Combustion,Transportation,Electric Power,Industrial,Residential,Commercial,U.S. Territories,Non-Energy Use of Fuels,Petroleum Systems,...,"Unspecified Mix of HFCs, PFCs, SF6, and NF3",Electronics Industry,NaN,Notes: Totals may not sum due to independent rounding. Parentheses indicate negative values or sequestration.,+ Does not exceed 0.5 kt.,M - Mixture of multiple gases,"a Emissions from Wood Biomass, Ethanol, and Biodiesel Consumption are not included specifically in summing Energy sector totals. Net carbon fluxes from changes in biogenic carbon reservoirs are accounted for in the estimates for LULUCF.",b Emissions from International Bunker Fuels are not included in totals.,c LULUCF emissions of LULUCF CH4 and N2O are reported separately from gross emissions totals. Refer to Table 2-8 for a breakout of emissions and removals for LULUCF by gas and source category.,d Small amounts of PFC emissions also result from this source.
year,,,,,,,,,,,,,,,,,,,,,
1990,"5,113,455","4,731,466","1,469,116","1,819,951","853,808","338,578","228,298","21,715","112,766","9,709",...,M,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1991,"5,057,931","4,680,858","1,422,498","1,817,359","839,111","347,719","232,665","21,505","118,155","10,000",...,M,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1992,"5,167,481","4,779,991","1,477,783","1,830,688","866,425","353,989","228,049","23,059","124,568","9,962",...,M,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993,"5,267,343","4,893,308","1,508,743","1,906,172","865,292","366,091","222,991","24,020","113,501","10,021",...,M,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1994,"5,358,824","4,964,870","1,554,070","1,930,468","872,851","357,050","224,700","25,732","124,026","10,274",...,M,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1995,"5,421,502","5,022,522","1,580,796","1,947,173","889,939","353,134","227,789","23,691","121,447","10,739",...,M,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996,"5,610,582","5,214,331","1,627,475","2,020,243","922,940","383,474","237,558","22,641","118,574","11,153",...,M,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997,"5,686,465","5,274,912","1,643,618","2,087,520","917,785","365,033","237,819","23,137","129,352","11,559",...,M,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998,"5,731,050","5,320,581","1,679,024","2,176,555","890,758","331,610","220,118","22,516","134,503","11,476",...,M,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Looking at the data which are not numbers
ghg_emissions_and_sinks.iloc[21].iloc[83]

'+'

In [5]:
# Looking at the data which are not numbers
ghg_emissions_and_sinks.iloc[27].iloc[83]

'+ '

In [6]:
# Looking at the data which are not numbers
ghg_emissions_and_sinks.iloc[29].iloc[77]

'M'

In [7]:
# Making non-numbers into blanks
ghg_emissions_and_sinks = ghg_emissions_and_sinks.replace('+', '')
ghg_emissions_and_sinks = ghg_emissions_and_sinks.replace('+ ', '')
ghg_emissions_and_sinks = ghg_emissions_and_sinks.replace('M', '')
ghg_emissions_and_sinks = ghg_emissions_and_sinks.replace('%M%', '')
ghg_emissions_and_sinks

Unnamed: 1,CO2,Fossil Fuel Combustion,Transportation,Electric Power,Industrial,Residential,Commercial,U.S. Territories,Non-Energy Use of Fuels,Petroleum Systems,...,"Unspecified Mix of HFCs, PFCs, SF6, and NF3",Electronics Industry,NaN,Notes: Totals may not sum due to independent rounding. Parentheses indicate negative values or sequestration.,+ Does not exceed 0.5 kt.,M - Mixture of multiple gases,"a Emissions from Wood Biomass, Ethanol, and Biodiesel Consumption are not included specifically in summing Energy sector totals. Net carbon fluxes from changes in biogenic carbon reservoirs are accounted for in the estimates for LULUCF.",b Emissions from International Bunker Fuels are not included in totals.,c LULUCF emissions of LULUCF CH4 and N2O are reported separately from gross emissions totals. Refer to Table 2-8 for a breakout of emissions and removals for LULUCF by gas and source category.,d Small amounts of PFC emissions also result from this source.
year,,,,,,,,,,,,,,,,,,,,,
1990,"5,113,455","4,731,466","1,469,116","1,819,951","853,808","338,578","228,298","21,715","112,766","9,709",...,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1991,"5,057,931","4,680,858","1,422,498","1,817,359","839,111","347,719","232,665","21,505","118,155","10,000",...,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1992,"5,167,481","4,779,991","1,477,783","1,830,688","866,425","353,989","228,049","23,059","124,568","9,962",...,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993,"5,267,343","4,893,308","1,508,743","1,906,172","865,292","366,091","222,991","24,020","113,501","10,021",...,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1994,"5,358,824","4,964,870","1,554,070","1,930,468","872,851","357,050","224,700","25,732","124,026","10,274",...,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1995,"5,421,502","5,022,522","1,580,796","1,947,173","889,939","353,134","227,789","23,691","121,447","10,739",...,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996,"5,610,582","5,214,331","1,627,475","2,020,243","922,940","383,474","237,558","22,641","118,574","11,153",...,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997,"5,686,465","5,274,912","1,643,618","2,087,520","917,785","365,033","237,819","23,137","129,352","11,559",...,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998,"5,731,050","5,320,581","1,679,024","2,176,555","890,758","331,610","220,118","22,516","134,503","11,476",...,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Deleting the columns with no data
ghg_emissions_and_sinks = ghg_emissions_and_sinks.dropna(axis='columns',how='any')

ghg_emissions_and_sinks   

Unnamed: 1,CO2,Fossil Fuel Combustion,Transportation,Electric Power,Industrial,Residential,Commercial,U.S. Territories,Non-Energy Use of Fuels,Petroleum Systems,...,Aluminum Production,Substitution of Ozone Depleting Substances d,SF6,Electrical Transmission and Distribution,Magnesium Production and Processing,Electronics Industry,NF3,Electronics Industry,"Unspecified Mix of HFCs, PFCs, SF6, and NF3",Electronics Industry
year,,,,,,,,,,,,,,,,,,,,,
1990,"5,113,455","4,731,466","1,469,116","1,819,951","853,808","338,578","228,298","21,715","112,766","9,709",...,,,1,1,,,,,,
1991,"5,057,931","4,680,858","1,422,498","1,817,359","839,111","347,719","232,665","21,505","118,155","10,000",...,,0,1,1,,,,,,
1992,"5,167,481","4,779,991","1,477,783","1,830,688","866,425","353,989","228,049","23,059","124,568","9,962",...,,0,1,1,,,,,,
1993,"5,267,343","4,893,308","1,508,743","1,906,172","865,292","366,091","222,991","24,020","113,501","10,021",...,,0,1,1,,,,,,
1994,"5,358,824","4,964,870","1,554,070","1,930,468","872,851","357,050","224,700","25,732","124,026","10,274",...,,,1,1,,,,,,
1995,"5,421,502","5,022,522","1,580,796","1,947,173","889,939","353,134","227,789","23,691","121,447","10,739",...,,,1,1,,,,,,
1996,"5,610,582","5,214,331","1,627,475","2,020,243","922,940","383,474","237,558","22,641","118,574","11,153",...,,,1,1,,,,,,
1997,"5,686,465","5,274,912","1,643,618","2,087,520","917,785","365,033","237,819","23,137","129,352","11,559",...,,,1,1,,,,,,
1998,"5,731,050","5,320,581","1,679,024","2,176,555","890,758","331,610","220,118","22,516","134,503","11,476",...,,,1,1,,,,,,


In [12]:
#Convert the the Index (year) into an integer
ghg_emissions_and_sinks.index = ghg_emissions_and_sinks.index.astype(int)

In [13]:
#Checking if the Index (year) is an integer
ghg_emissions_and_sinks.index

Int64Index([1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000,
            2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011,
            2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019],
           dtype='int64', name='year')

In [14]:
#output_data_file = "honey_vs_polls.csv"  
ghg_emissions_and_sinks.to_csv("ghg_emissions_cleaned.csv", header=True)